<a href="https://colab.research.google.com/github/Gtarcraz/botorch_test/blob/master/Introduction_to_Botorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BoTorch Tutorials

The following information is summarised from the main BoTorch website.
https://botorch.org/docs/introduction.html

I have decided to summarise it to extract the core information from the documents to get started.


## Introduction to BoTorch

BoTorch is a library for Bayesian Optimisation research built on top of PyTorch. 

Bayesian Optimisation (BayesOpt) is an established technique for sequential optimisation of costly-to-evaluate black box functions. It can be applied to a wide variety of problems, including hyperparameter optimisation for machine learning algorithms, A/B testing, as well as many scientific and engineering problems.

## Why Botorch?

1. Improved Developer Efficiency
--> Modular and easily extensible interface for composing Bayesian Optimisation primitives (include probabilitic mdels, acquisition functions and optimisers).
--> Utilise quasi-Monte-Carlo acquisition functions

2. State-of-the-art Modelling
--> Provide support for state-of-the-art probabilisitc models in GPyTorch (Library for efficient, scalable Gaussian Process implemented in PyTorch).
--> Features include multi-task GPs, deep kernel learning, deep GPs, and approximate inference

3. Harnessing the features of PyTorch
--> Auto-differentiation, GPU implementations and dynamic computation graph 



# Simple Programme to get started

In [1]:
# ===============
# Install BoTorch
# ===============

# Via conda
# conda install botorch -c pytorch -c gpytorch

# Via pip
!pip install botorch

     |████████████████████████████████| 225kB 8.4MB/s 
     |████████████████████████████████| 245kB 14.4MB/s 
  Created wheel for gpytorch: filename=gpytorch-1.1.1-py2.py3-none-any.whl size=400467 sha256=69c31a85387f48ab60cd0be642d3a40b7eef8ed5f2eb0b505c0825f9d31fdc9d
  Stored in directory: /root/.cache/pip/wheels/e5/a5/29/4dafc0624adf678108e0067836556f0c72588e85d851d78ae0
Successfully built gpytorch


In [3]:
# ===============
# Fitting a Model
# ===============

import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_model
from botorch.utils import standardize
from gpytorch.mlls import ExactMarginalLogLikelihood


# Create training set
train_X = torch.rand(10,2)  # Create random matrix 10 rows x 2 columns
print("Training X \n\f\n", train_X)
Y = 1-torch.norm(train_X,dim=-1,keepdim=True) # Generate Y = 1-sqrt(X1^2+X2^2)
print("\nY\n\f\n", Y)
Y = Y + 0.1*torch.randn_like(Y)  # Add some noise (Size of Y)
print("\nY with noise\n\f\n", Y)
train_Y = standardize(Y) # Stadardisation of Y for training

gp = SingleTaskGP(train_X,train_Y)
mll = ExactMarginalLogLikelihood(gp.likelihood,gp)
fit_gpytorch_model(mll)

Training X 

 tensor([[0.8223, 0.1913],
        [0.7043, 0.2498],
        [0.9731, 0.0981],
        [0.0613, 0.0958],
        [0.1632, 0.7125],
        [0.6452, 0.9783],
        [0.4497, 0.3366],
        [0.9503, 0.8316],
        [0.0662, 0.3149],
        [0.9382, 0.1787]])

Y

 tensor([[0.8442],
        [0.7473],
        [0.9781],
        [0.1137],
        [0.7310],
        [1.1719],
        [0.5617],
        [1.2628],
        [0.3218],
        [0.9550]])

Y with noise

 tensor([[0.8394],
        [0.6034],
        [0.8910],
        [0.0109],
        [0.7117],
        [1.3695],
        [0.4605],
        [1.3722],
        [0.2963],
        [1.1272]])


ExactMarginalLogLikelihood(
  (likelihood): GaussianLikelihood(
    (noise_covar): HomoskedasticNoise(
      (noise_prior): GammaPrior()
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
  )
  (model): SingleTaskGP(
    (likelihood): GaussianLikelihood(
      (noise_covar): HomoskedasticNoise(
        (noise_prior): GammaPrior()
        (raw_noise_constraint): GreaterThan(1.000E-04)
      )
    )
    (mean_module): ConstantMean()
    (covar_module): ScaleKernel(
      (base_kernel): MaternKernel(
        (lengthscale_prior): GammaPrior()
        (raw_lengthscale_constraint): Positive()
        (distance_module): Distance()
      )
      (outputscale_prior): GammaPrior()
      (raw_outputscale_constraint): Positive()
    )
  )
)